# Spectral Clustering for Connectivity Matrices

### Introduction
This code uses `sklearn` to calculate the Spectral Clustering of a similarity matrix S.
<br>First let us import the appropriate modules

In [ ]:
from scipy.io import loadmat    # needed for loading .mat matlab files
import numpy as np              # for vector math
import os                       # for getting paths to files in your directory
import sklearn.cluster          # for spectral clustering (and other clustering methods)
import matplotlib.pyplot as plt # for plotting (in general)
from nilearn import plotting    # for plotting the brain (nilearn = neuroimaging)
import pandas as pd             # for easier data manipulation

## Define functions
Then create a function to find files in the `./data/` subfolder

In [6]:
def get_path(filename):
    """Find filename in ./data/ directory.

    Args:
        filename (str): file we're looking for in the ./data/ directory.

    Returns:
        str: path to file "filename" in ./data/ dir.

    """

    here_dir = os.path.dirname(os.path.realpath('__file__'))
    file_dir = os.path.join(here_dir, 'data', filename)
    return(file_dir)

One to return the connectivity matrices:

In [7]:
def get_connectivity_matrices(filepath):
    """Read connectivity matrices from .mat file.

    Args:
        filepath (str): full path to .mat file containing connectivity matrices.

    Returns:
        arr: Array of connectivity matrices, one per patient.

    """

    from pathlib import Path
    my_file = Path(filepath)

    if my_file.is_file():
        healthy_14    = loadmat(my_file)['S'] #load data for all 14 patients
        conn_matrices = np.transpose(healthy_14)
        return(conn_matrices)

    else:
        print(my_file)
        raise(FileNotFoundError('Is path to file correct??'))

And finally one to plot the glass brains using `nilearn`

In [8]:
def plot_glass_brains(color):
    """Plot a glass brain for a 90 regions ATLAS with nodes colored by `color`.

    Args:
        color (list): Color indices. e.g. [0,1,1,0] will color nodes [1,2] differently.

    Returns:
        matplotlib.plot: A plot object.

    """

    coordinates_path = get_path('coordinates.csv')
    df = pd.read_csv(coordinates_path)
    color = color

    coords = [(df['X'][i],df['Y'][i],df['Z'][i]) for i in range(90)]
    connec = np.array([[0]*90]*90)

    plotting.plot_connectome(connec, coords, node_color=color, display_mode='lyrz')

# Calculate spectral clustering
Now we are ready to start. We will find and define a connectivity matrix first:

In [35]:
conn_matrices_dir = get_path('connectivity_matrices.mat')        # find .mat file containing connectivity matrices
conn_matrices     = get_connectivity_matrices(conn_matrices_dir) # connectivity matrices for 14 healthy subjects

Spectral clustering works by finding elements **near** to each other. We however want to cluster brain regions based on their connectivity, which is **largest** for highly connected regions. We therefore have to do 2 things:
1. Invert connectivity matrix so that **large** values (highly connected regions) become near zero (close in 'similarity' space
2. Normalize the matrix so that values are between [0,1]. We will use a Gaussian kernel for that as suggested in [here](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html)

In [39]:
shifted_matrices    = conn_matrices + 0.0001 #we will invert the connectivity matrix (1/M) so lets get rid of zeros
similarity_matrices = (1/ shifted_matrices) #similarity matrices are the inverted connectivity matrices

In [40]:
gauss_parameter     = 0.005
kernelized_matrices = np.exp(- similarity_matrices ** 2 / (2. * gauss_parameter ** 2))

We then define the `Spectral Clustering` function that will be applied to the data and apply it

In [41]:
spectral   = sklearn.cluster.SpectralClustering(n_clusters=2, affinity='precomputed') #we choose to classify the data into 2 clusters
patient_ID = 0
spectral.fit(kernelized_matrices[patient_ID])

SpectralClustering(affinity='precomputed', assign_labels='kmeans', coef0=1,
          degree=3, eigen_solver=None, eigen_tol=0.0, gamma=1.0,
          kernel_params=None, n_clusters=2, n_init=10, n_jobs=None,
          n_neighbors=10, random_state=None)

Now we will use the labels, created by the `Spectral Clustering` method, to color the nodes in the glass brain differently

In [44]:
color = spectral.labels_ #array of integer labels referrent to which class the node belongs to

In [ ]:
plot_glass_brains(color)